In [ ]:
from bs4 import BeautifulSoup
import urllib.request

url =  'https://www.vinmec.com/vi/tin-tuc/hoi-dap-bac-si/?page=5'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'html.parser')


In [ ]:
from tqdm import tqdm
web_url = []
image_url = []
for i in tqdm(range(1,563)):
  url =  f'https://www.vinmec.com/vi/tin-tuc/hoi-dap-bac-si/?page={i}'
  page = urllib.request.urlopen(url)
  soup = BeautifulSoup(page, 'html.parser')
  content = (soup.find_all("div",class_ = "post-list")[0]).find_all("a")
  for web_url_raw in content:
    web_url.append(web_url_raw.get("href")) if web_url_raw.get("href") is not None else web_url.append("")
    img_tag = web_url_raw.find("img")
    image_url.append(web_url_raw.find("img").get("src")) if img_tag is not None else image_url.append("")


In [ ]:
import re
from bs4 import BeautifulSoup
import urllib.request
def data_clean_in_vinmect_web(url):
  page = urllib.request.urlopen(url)
  soup = BeautifulSoup(page, 'html.parser')
  new_feed = soup.find_all("div",class_ = "rich-text")
  search  = re.search("</b></p><p><i>([\w+\W+]+)</i></p><p><b>",str(new_feed[0]))

  question = search.groups()[0]
  question = re.sub("</i></p><p><i>|</i><b><i>|</i></b><i>|</p><p><i>|</i></p><p><b>|</i> <b><i>",'',question)
  all_link = re.search("Trả lời</b></p><p>(.+)</p><p>",str(new_feed[0]))
  anwer = all_link.groups()[0]
  regex_pattern = r'<a href|<a>|><b>|</b></a>'

  # Split the original string based on the pattern
  split_string = re.split(regex_pattern, anwer)
  split_string_copy = split_string
  links = []
  for idx,string in enumerate(split_string):
    if '="https:' in string:
      split_string_copy[idx] = "[link]"
      links.append(re.sub('=|"','',string))
  response = ''.join(split_string_copy)
  answer = re.sub("</p><p>| &gt;|<b>|</b>",'',response)
  return question,answer,links

In [ ]:
import pandas as pd
from tqdm import tqdm
data2 = pd.read_csv("vinmect_url.csv")
datax  = pd.DataFrame({"urls":[None]*5065,"image_title":[None]*5065,"question":[None]*5065,"answer":[None]*5065,"img_link":[None]*5065})

for i in tqdm(range(3248,5056)):
  url = "https://www.vinmec.com/vi"+data2['url'][i]
  try:
    question,answer,links = data_clean_in_vinmect_web(url)
  except:
    question,answer,links = ["","",""]
  datax.iloc[i,:] = [data2.iloc[i,0],data2.iloc[i,1],question,answer,links]
  datax.to_csv("vinmect_crawling.csv",index =False)